In [16]:
"""
This scipr runs a stripped version of the DDDurban, where subsurface moisture states are drwn from a distribuiton.
In addition, extreme precipitation of specidfied duration and the temporal distribuion of that precipitation for shorter
timesteps than the duration are dran from a GPD distribution and a beta distribuion. Short timesseries a re simulated many times
so that a extremevalue distribiion of floods can be build
"""

using LsqFit
using Statistics
using Dates
using DataFrames
using Plots
using CSV
using BlackBoxOptim
using Distributions

##Preprocessing routines
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\Big2SmallLambda.jl")
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\CeleritySubSurface.jl")
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\SingleUH.jl")
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\SingleNormalUH.jl")
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\LayerEstimationDesign.jl")
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\GrvInputDistributionICap2022.jl")
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\OFICap.jl")
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\LayerCapacityUpdate.jl")
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\LayerCapacityInit.jl")
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\LayerUpdate.jl")
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\LayerInitUrbanDesign.jl")
include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\RiverUpdate.jl")


include("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Julia\\DDDFunctions\\DDDEventFloodDesign.jl")
########################################################################################


t1= time_ns()

catchment = "123.38"#
paramfile = string("\\\\nve.no\\fil\\h\\HM\\Interne Prosjekter\\UrbanDesignFlom\\Parameters\\",catchment,"\\Par_",
    catchment,"_3(15)_AAR.csv")
#utfile  = string("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Urban\\utdata\\Risvollan\\simres_",
#    catchment,"_ths_FD_SON_3(15).csv")
utfile  = string("\\\\nve.no\\fil\\h\\HM\\Interne Prosjekter\\UrbanDesignFlom\\utdata\\Risvollan\\simres_",
    catchment,"_AAR_3(15)_SScontrib.csv")
#utfile2  = string("\\\\nve.no\\fil\\h\\HB\\HB-modellering\\DDDtestbenk\\DDD_Urban\\utdata\\Risvollan\\simres2_",
#    catchment,"_ths_FD_SON_3(15).csv")
utfile2  = string("\\\\nve.no\\fil\\h\\HM\\Interne Prosjekter\\UrbanDesignFlom\\utdata\\Risvollan\\simres2_",
    catchment,"_ARR_3(15)_SScontrib.csv")

prm = CSV.read(paramfile,DataFrame,header=["Name", "val"], delim=';')    
# Parameters to be calibrated
#         OFP,         GscInt     persons       GPsh         ICapP
tprm = [prm.val[19], prm.val[22], prm.val[24], prm.val[28], prm.val[31]]

#Gshape, Gscale = Big2SmallLambda(prm.val[22], prm.val[22]) # GshInt and GscInt Converts integrated celerity to layers. 

NumSim = 100000
 
function calib_wrapper_model(tprm, prm, utfile, utfile2, NumSim)
 maxP = DDDEventFloodDesign(tprm, prm, utfile, utfile2, NumSim)  
 return maxP 
end
    
calib_wrapper_model(tprm, prm, utfile, utfile2, NumSim) # a single run     

t2 = time_ns()
println("Time elapsed[s]= ",(t2-t1)/1.0e9)

   

Number of timesteps in RN= 3
Location par GDP = 2.22
Scale par GDP= 1.25
Shape par GPD= 0.19
Time elapsed[s]= 177.6949634
